In [ ]:
import cv2
import numpy as np
import os
import re
import math
import piexif
import shutil
import xml.etree.ElementTree as ET

def return_extracted(value):
    """Extract numeric values from an XMP metadata field."""
    return float(re.sub(r'[^\d.-]', '', value))

def extract_xmp_data(image_path):
    """Extracts Gimbal PRY values from XMP metadata of an image."""
    with open(image_path, 'rb') as f:
        data = f.read()

    xmp_start = data.find(b'<x:xmpmeta')
    xmp_end = data.find(b'</x:xmpmeta>')

    if xmp_start != -1 and xmp_end != -1:
        xmp_end += len(b'</x:xmpmeta>')
        try:
            xmp_str = data[xmp_start:xmp_end].decode('utf-8', errors='replace')
            root = ET.fromstring(xmp_str)

            # Initialize PRY variables
            gimbal_pitch, gimbal_roll, gimbal_yaw = None, None, None

            # Extract PRY values
            for elem in root.iter():
                for key, value in elem.attrib.items():
                    if "GimbalPitchDegree" in key:
                        gimbal_pitch = return_extracted(value)
                    elif "GimbalRollDegree" in key:
                        gimbal_roll = return_extracted(value)
                    elif "GimbalYawDegree" in key:
                        gimbal_yaw = return_extracted(value)

            if gimbal_pitch is not None and gimbal_roll is not None and gimbal_yaw is not None:
                return gimbal_pitch, gimbal_roll, gimbal_yaw
            else:
                return None  # PRY values not found
        except Exception as e:
            print(f"[ERROR] Decoding XMP metadata from {image_path}: {e}")
            return None
    else:
        return None  # No XMP metadata found

def extract_exif_data(image_path):
    """Extract EXIF metadata using piexif."""
    try:
        exif_data = piexif.load(image_path)
        return exif_data
    except Exception as e:
        print(f"[ERROR] Failed to extract EXIF from {image_path}: {e}")
        return None

def get_adjusted_rotation_matrix(angle, width, height):
    """Compute rotation matrix and adjust image size to prevent cropping."""
    center = (width / 2, height / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    cos = abs(rotation_matrix[0, 0])
    sin = abs(rotation_matrix[0, 1])

    new_width = int((height * sin) + (width * cos))
    new_height = int((height * cos) + (width * sin))

    rotation_matrix[0, 2] += (new_width / 2) - center[0]
    rotation_matrix[1, 2] += (new_height / 2) - center[1]

    return rotation_matrix, new_width, new_height

def correct_image_perspective(image_path, output_path, summary):
    """Correct an image's perspective using PRY values to get NADIR and preserve metadata."""
    pry_values = extract_xmp_data(image_path)
    exif_data = extract_exif_data(image_path)

    if pry_values:
        pitch, roll, yaw = pry_values
        print(f"\n📸 Processing: {os.path.basename(image_path)}")
        print(f"   📌 Current PRY → Pitch: {pitch}°, Roll: {roll}°, Yaw: {yaw}°")

        # **✅ Skip images with pitch < -93° OR pitch > -87°**
        if pitch < -93 or pitch > -87:
            summary["skipped"] += 1
            print(f"   ⏩ Skipping image due to pitch being outside [-93°, -87°].\n")
            return

        # **✅ If already at NADIR (-90, 0, 0), copy it directly to output**
        if abs(pitch + 90) == 0 and abs(roll) == 0 and abs(yaw) == 0:
            shutil.copy(image_path, os.path.join(output_path, os.path.basename(image_path)))
            summary["nadir_copied"] += 1
            print(f"   ✅ Image already at NADIR. Copied to output folder.\n")
            return

        # Read the image
        image = cv2.imread(image_path)
        h, w = image.shape[:2]

        # Compute rotation matrices with adjusted output size
        M_yaw, new_w_yaw, new_h_yaw = get_adjusted_rotation_matrix(-yaw, w, h)
        M_roll, new_w_roll, new_h_roll = get_adjusted_rotation_matrix(-roll, new_w_yaw, new_h_yaw)
        M_pitch, new_w_pitch, new_h_pitch = get_adjusted_rotation_matrix(-(pitch + 90), new_w_roll, new_h_roll)

        # Apply rotations in sequence: Yaw → Roll → Pitch (to achieve NADIR)
        rotated_image = cv2.warpAffine(image, M_yaw, (new_w_yaw, new_h_yaw))
        rotated_image = cv2.warpAffine(rotated_image, M_roll, (new_w_roll, new_h_roll))
        rotated_image = cv2.warpAffine(rotated_image, M_pitch, (new_w_pitch, new_h_pitch))

        # Save the corrected image
        corrected_filename = os.path.join(output_path, os.path.basename(image_path))
        cv2.imwrite(corrected_filename, rotated_image)

        # Restore EXIF metadata
        if exif_data:
            piexif.insert(piexif.dump(exif_data), corrected_filename)

        summary["corrected"] += 1
        print(f"   ✅ Corrected & Saved → {corrected_filename}\n")

    else:
        summary["errors"] += 1
        print(f"[ERROR] PRY extraction failed for {image_path}. Skipping...")

def process_folder(folder_path, output_path):
    """Process all images in a folder and correct them to NADIR view, preserving metadata."""
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff', '.dng'))]

    if not image_files:
        print("[ERROR] No image files found in the folder.")
        return

    summary = {"total": len(image_files), "nadir_copied": 0, "corrected": 0, "skipped": 0, "errors": 0}

    print(f"\n📂 Processing {summary['total']} images in folder: `{folder_path}`\n")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        correct_image_perspective(image_path, output_path, summary)

    # **Print Final Summary**
    print("\n📊 **Processing Summary**:")
    print(f"   🔹 **Total Images Processed:** {summary['total']}")
    print(f"   ✅ **Already NADIR (Copied):** {summary['nadir_copied']}")
    print(f"   🔄 **Corrected & Saved:** {summary['corrected']}")
    print(f"   ⏩ **Skipped Due to Pitch < -100° or > -80°:** {summary['skipped']}")
    print(f"   ❌ **Errors Encountered:** {summary['errors']}\n")

# 🔹 Define input and output folders
input_folder = "Data"
output_folder = "main_output"

# 🔥 Run the script
process_folder(input_folder, output_folder)



📂 Processing 400 images in folder: `Data`


📸 Processing: DJI_20240614145034_0002.JPG
   📌 Current PRY → Pitch: -40.2°, Roll: 0.0°, Yaw: -1.2°
   ✅ Corrected & Saved → main_output\DJI_20240614145034_0002.JPG


📸 Processing: DJI_20240614145048_0005_HA-waypoint2-80m.JPG
   📌 Current PRY → Pitch: -66.1°, Roll: 0.0°, Yaw: -179.8°
   ✅ Corrected & Saved → main_output\DJI_20240614145048_0005_HA-waypoint2-80m.JPG


📸 Processing: DJI_20240614145056_0006_HA-waypoint2-80m.JPG
   📌 Current PRY → Pitch: -63.8°, Roll: 0.0°, Yaw: -179.8°
   ✅ Corrected & Saved → main_output\DJI_20240614145056_0006_HA-waypoint2-80m.JPG


📸 Processing: DJI_20240614145104_0007_HA-waypoint2-80m.JPG
   📌 Current PRY → Pitch: -89.9°, Roll: 180.0°, Yaw: 0.2°
   ✅ Corrected & Saved → main_output\DJI_20240614145104_0007_HA-waypoint2-80m.JPG


📸 Processing: DJI_20240614145111_0008_HA-waypoint2-80m.JPG
   📌 Current PRY → Pitch: -89.9°, Roll: 180.0°, Yaw: 0.2°
   ✅ Corrected & Saved → main_output\DJI_20240614145111_0008_HA-way